In [33]:
# Import Statemements

import random
import pickle # used to save and restore python objects
import gzip
import numpy as np
import tensorflow as tf
from keras import backend as K

EPOCHS = 5
RESIZE_WIDTH = 28
RESIZE_HEIGHT = 28

In [19]:
# Load the Dataset

f = gzip.open('./data/mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding = 'latin1')
f.close()

In [20]:
# Function: to_categorical -- One-Hot Vector Encoding 

# Converts class labels (integers from 0 to nb_classes) to one-hot vector
# Example: 5 => [0 0 0 0 0 1 0 0 0 0]

# Arguments:
    # y: array, class labels to convert
    # nb_classes: integer, total number of classes 

def to_categorical(y, nb_classes):
    y = np.asarray(y, dtype='int32')
    if not nb_classes:
        nb_classes = np.max(y)+1
    answer = np.zeros((len(y), nb_classes))
    answer[np.arange(len(y)),y] = 1.
    return answer

In [25]:
# Train-Test Split, One-Hot Encoding for our data

train_x = train_set[0]
train_y = to_categorical(train_set[1], 10)
test_x = test_set[0]
test_y = to_categorical(test_set[1],10)

if K.image_data_format() == 'channels_first':
    train_x = train_x.reshape(train_x.shape[0], 1, RESIZE_WIDTH, RESIZE_HEIGHT)
    test_x = test_x.reshape(test_x.shape[0], 1, RESIZE_WIDTH, RESIZE_HEIGHT)
    input_shape = (1, RESIZE_WIDTH, RESIZE_HEIGHT)
else:
    train_x = train_x.reshape(train_x.shape[0], RESIZE_WIDTH, RESIZE_HEIGHT, 1)
    test_x = test_x.reshape(test_x.shape[0], RESIZE_WIDTH, RESIZE_HEIGHT, 1)
    input_shape = (RESIZE_WIDTH, RESIZE_HEIGHT, 1)

# Result of One-Hot Encoding Class Labels 
print(test_y[:5])

print('Train set has dimensionality %s' % str(train_x.shape))
print('Test set has dimensionality %s' % str(test_x.shape))

# Apply some normalization here.
train_x = train_x.astype('float32')
test_x = test_x.astype('float32')
train_x /= 255
test_x /= 255

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Train set has dimensionality (50000, 28, 28, 1)
Test set has dimensionality (10000, 28, 28, 1)


In [31]:
######################################
# Import necessary layers.
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm
######################################

model = Sequential()

######################################
# Define the network
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=input_shape))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, kernel_initializer='glorot_normal'))
model.add(Activation('softmax'))
######################################


######################################
# Define your loss and your objective
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
######################################

In [32]:
######################################
# Define the batch size
batch_size = 64
######################################
model.fit(train_x, train_y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 270s 7ms/step - loss: 0.2628 - acc: 0.9117 - val_loss: 0.1156 - val_acc: 0.9578
Epoch 2/5
40000/40000 [==============================] - 273s 7ms/step - loss: 0.0947 - acc: 0.9652 - val_loss: 0.0644 - val_acc: 0.9769
Epoch 3/5
40000/40000 [==============================] - 274s 7ms/step - loss: 0.0644 - acc: 0.9773 - val_loss: 0.0490 - val_acc: 0.9825
Epoch 4/5
40000/40000 [==============================] - 263s 7ms/step - loss: 0.0502 - acc: 0.9827 - val_loss: 0.0404 - val_acc: 0.9859
Epoch 5/5
40000/40000 [==============================] - 273s 7ms/step - loss: 0.0413 - acc: 0.9859 - val_loss: 0.0362 - val_acc: 0.9867


In [35]:
# Evaluating Performance
loss, acc = model.evaluate(test_x, test_y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))

10000/10000 [==============================] - 20s 2ms/step

Got 98.94% accuracy
